# Setup the dataframe

In [1]:
import pandas as pd
import os
from datetime import datetime, timezone
import time
import matplotlib.pyplot as plt

all_data = pd.DataFrame()

# Be careful with this. If there is more than one CSV file only the last one will be used. 
for file in os.listdir('./'):
    if file.endswith('csv'):
        all_data = pd.read_csv('./' + file, sep=';')
        print("Using file: " + file)


c:\users\benjj\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (643,644,645,646,647,648,649) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Using file: 2020_09_16_14_02_20.csv


## Setup the Date_Time_(GMT) column

In [2]:
start = time.time()

all_data['PLC_TIME(Timedate48)'] = all_data['PLC_TIME(Timedate48)'].astype(str)

def getUnixTime(date):
    return date.split(',')[0]

def convertToUTC(date):
    return datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')

def convertToGMT(unix_timestamp):
    utcTime = datetime.fromtimestamp(unix_timestamp, timezone.utc)
    localTime = utcTime.astimezone()
    return localTime.strftime("%Y-%m-%d %H:%M:%S")
    
all_data['Date_Time_(GMT)'] = all_data['PLC_TIME(Timedate48)'].apply(lambda x: getUnixTime(x))
all_data['Date_Time_(GMT)'] = all_data['Date_Time_(GMT)'].astype('int32')
all_data['Date_Time_(GMT)'] = all_data['Date_Time_(GMT)'].apply(lambda x: convertToGMT(x))
column_name = 'Date_Time_(GMT)'
first_column = all_data.pop(column_name)
all_data.insert(0, column_name, first_column)
all_data['Date_Time_(GMT)'] = pd.to_datetime(all_data['Date_Time_(GMT)'])
all_data = all_data.sort_values(by='Date_Time_(GMT)')

duration = time.time() - start
print("Total time: %10.2f" % duration)

all_data.tail()

Total time:       1.79


,Date_Time_(GMT),TIME,PLC_TIME_OFFSET(Integer8),PLC_TIME_CV(Enum2),PLC_TIME(Timedate48),EVR_VehicleNumber(Unsigned16),EVR_TIME_CV(Enum2),EVR_TIME(Timedate48),EVR_Speed(Unsigned16),EVR_Speed_CV(Enum2),...,EKE_DIOs_Car15(Bitset32),EKE_DIOs_Car16(Bitset32),EKE_DIOs_Car2(Bitset32),EKE_DIOs_Car3(Bitset32),EKE_DIOs_Car4(Bitset32),EKE_DIOs_Car5(Bitset32),EKE_DIOs_Car6(Bitset32),EKE_DIOs_Car7(Bitset32),EKE_DIOs_Car8(Bitset32),EKE_DIOs_Car9(Bitset32)
32496,2020-09-16 18:39:38,"16637,952",0,1,"1600277978,000",15002,1,"1600277978,000",0,1,...,0,0,31c083,31c083,31c083,31c083,31c083,31c083,431c083,0
32497,2020-09-16 18:39:39,"16638,464",0,1,"1600277979,000",15002,1,"1600277979,000",0,1,...,0,0,31c083,31c083,31c083,31c083,31c083,31c083,431c083,0
32498,2020-09-16 18:39:39,"16638,976",0,1,"1600277979,000",15002,1,"1600277979,000",0,1,...,0,0,31c083,31c083,31c083,31c083,31c083,31c083,431c083,0
32499,2020-09-16 18:39:40,"16639,488",0,1,"1600277980,000",15002,1,"1600277980,000",0,1,...,0,0,31c083,31c083,31c083,31c083,31c083,31c083,431c083,0
32500,2020-09-16 18:39:40,"16640,000",0,1,"1600277980,000",15002,1,"1600277980,000",0,1,...,0,0,31c083,31c083,31c083,31c083,31c083,31c083,431c083,0


#### Cleanup the NaN entries

In [ ]:
print("Rows before: " + str(all_data.shape[0]))
all_data = all_data.dropna(how='all')
print("Rows after: " + str(all_data.shape[0]))
all_data.head()

In [4]:
HABD_columns = [col for col in all_data if not 'Temp' in col or 'Alarm' in col]
HABD_columns.remove('Date_Time_(GMT)')
HABD_columns.remove('EVR_VehicleNumber(Unsigned16)')
HABD_columns.remove('EVR_Speed(Unsigned16)')
HABD_columns.remove('EVR_Speed_CV(Enum2)')
HABD_columns.append('HABD_HighTemp(Unsigned16)')

print("cols in list: " + str(len(HABD_columns)))
print("cols in all_data: " + str(all_data.shape[1]))
print("rows in all_data: " + str(all_data.shape[0]))

cols in list: 374
cols in all_data: 652
rows in all_data: 32501


In [5]:
before = all_data.shape[1]
#all_data.drop(HABD_columns, axis=1)
for column in HABD_columns:
    del all_data[column]
# HABD_columns.pop(0) # removes useless column 115, 115 ...
print("Rows removed: " + str(before - all_data.shape[1]))
print("cols in all_data: " + str(all_data.shape[1]))
print("rows in all_data: " + str(all_data.shape[0]))

Rows removed: -374
cols in all_data: 278
rows in all_data: 32501


In [12]:
all_data.head(10)

,Date_Time_(GMT),EVR_VehicleNumber(Unsigned16),EVR_Speed(Unsigned16),EVR_Speed_CV(Enum2),HABD_HighTemp(Unsigned16),HABD_TempDiff(Boolean1),HABD_TempDiff_Car9(Bitset8),HABD_TempDiff_Car8(Bitset8),HABD_TempDiff_Car7(Bitset8),HABD_TempDiff_Car6(Bitset8),...,HABD_TempCh11_Car2(Integer16),HABD_TempCh11_Car1(Integer16),HABD_TempCh10_Car9(Integer16),HABD_TempCh10_Car8(Integer16),HABD_TempCh10_Car7(Integer16),HABD_TempCh10_Car6(Integer16),HABD_TempCh10_Car5(Integer16),HABD_TempCh10_Car4(Integer16),HABD_TempCh10_Car3(Integer16),HABD_TempCh10_Car2(Integer16)
0,2020-09-16 14:02:20,15002,0,1,115,0,0,0,0,0,...,20,19,0,19,19,19,19,18,19,20
1,2020-09-16 14:02:20,15002,0,1,115,0,0,0,0,0,...,20,19,0,19,19,19,19,18,19,20
2,2020-09-16 14:02:21,15002,0,1,115,0,0,0,0,0,...,20,19,0,19,19,19,19,18,19,20
3,2020-09-16 14:02:21,15002,0,1,115,0,0,0,0,0,...,20,19,0,19,19,19,19,18,19,20
4,2020-09-16 14:02:22,15002,0,1,115,0,0,0,0,0,...,20,19,0,19,19,19,19,18,19,20
5,2020-09-16 14:02:22,15002,0,1,115,0,0,0,0,0,...,20,19,0,19,19,19,19,18,19,20
6,2020-09-16 14:02:23,15002,0,1,115,0,0,0,0,0,...,20,19,0,19,19,19,19,18,19,20
7,2020-09-16 14:02:23,15002,0,1,115,0,0,0,0,0,...,20,19,0,19,19,19,19,18,19,20
8,2020-09-16 14:02:24,15002,0,1,115,0,0,0,0,0,...,20,19,0,19,19,19,19,18,19,20
9,2020-09-16 14:02:24,15002,0,1,115,0,0,0,0,0,...,20,19,0,19,19,19,19,18,19,20


### Remove 0 data columns

In [14]:
HABD_cols = [col for col in all_data if not 'Date_Time_(GMT)' in col]
before = all_data.shape[1]

for col in HABD_cols:
    if all_data[col].sum() == 0:
        del all_data[col]

print("Rows removed: " + str(before - all_data.shape[1]))
print("cols in all_data: " + str(all_data.shape[1]))

Rows removed: 146
cols in all_data: 132


### Check the data counts

In [16]:
HABD_cols = [col for col in all_data if not 'Date_Time_(GMT)' in col]

for col in HABD_cols[1:]:
    print(all_data[col].value_counts())

1    32501
Name: EVR_Speed_CV(Enum2), dtype: int64
115    32501
Name: HABD_HighTemp(Unsigned16), dtype: int64
18    22194
19    10307
Name: HABD_TempCh10_Car1(Integer16), dtype: int64
19    29794
0      2618
20       89
Name: HABD_TempCh9_Car8(Integer16), dtype: int64
20    29881
0      2620
Name: HABD_TempCh9_Car7(Integer16), dtype: int64
19    24384
20     5497
0      2620
Name: HABD_TempCh9_Car6(Integer16), dtype: int64
18    24097
19     5784
0      2620
Name: HABD_TempCh9_Car5(Integer16), dtype: int64
20    29881
0      2620
Name: HABD_TempCh9_Car4(Integer16), dtype: int64
19    24316
20     3894
0      2618
18     1673
Name: HABD_TempCh9_Car3(Integer16), dtype: int64
18    23324
19     6557
0      2620
Name: HABD_TempCh9_Car2(Integer16), dtype: int64
19    32501
Name: HABD_TempCh9_Car1(Integer16), dtype: int64
20    15093
21    14790
0      2618
Name: HABD_TempCh8_Car8(Integer16), dtype: int64
19    29792
0      2620
20       89
Name: HABD_TempCh8_Car7(Integer16), dtype: int64
20

### Filter out the low temperatures